In [ ]:
import syft as sy

In [ ]:
from syft.service.worker.worker_image import SyftWorkerImage, SyftWorkerImageTag
from syft.custom_worker.config import DockerWorkerConfig
from syft.service.worker.worker_image import build_using_docker

In [ ]:
domain  = sy.orchestra.launch(name="domain", reset=True, dev_mode=True)

In [ ]:
domain_client = domain.client.login(email="info@openmined.org", password="changethis")

In [ ]:
nginx_dockerfile_str = """
# Use the official Nginx image as the base
FROM nginx:latest

# Expose port 80 for the Nginx server
EXPOSE 80

# Start Nginx when the container has provisioned
CMD ["nginx", "-g", "daemon off;"]
"""

In [ ]:
docker_config = DockerWorkerConfig(nginx_dockerfile_str)

In [ ]:
print(docker_config.dockerfile)

In [ ]:
domain_client.api.services.worker_image.submit_dockerfile(docker_config=docker_config)

In [ ]:
res = domain_client.api.services.worker_image.list()

In [ ]:
res

In [ ]:
domain_client.api.services.worker_image.build(uid=res[0].id, tag="openmined/test-nginx:0.7.8")

In [ ]:
res = domain_client.api.services.worker_image.list()

In [ ]:
res[0].created_at.utc_timestamp

In [ ]:
res[0].config.dockerfile

In [ ]:
worker_image = SyftWorkerImage(
    config=docker_config,
    image_tag=SyftWorkerImageTag.from_str("openmined/grid-nginx:0.8.3"),
    created_by=domain_client.verify_key
)

In [ ]:
worker_image.image_tag

In [ ]:
output = build_using_docker(worker_image=worker_image)

In [ ]:
output